## Import Earth Engine API and authenticate<a class="anchor" id="import-api"></a>

In [1]:
import ee

## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=jYFl5BjfqVAoF4X0sxMnJ1PO6gHcYkLnFaafDJ26hQw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiVZPPWvoBrEoNs9H5ZnWd4PD1vFPfh0dFIhU1McTcUrPffNYHsd8I

Successfully saved authorization token.


## Install the geemap Python package

In [2]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 

geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


## Create Map object and isolate area of interest (aoi)

In [3]:
Map = geemap.Map()
Map.setCenter(-75.722793, 23.182838, 5)
#aoi = ee.Geometry.Polygon([[-88.20, 30.99], [-59.68, 20.06] , [-62.45, 8.09] , [-86.48, 14.29]]) #4 sided polygon of the Florida, Puerto Rico, Carribean area
aoi = ee.FeatureCollection('TIGER/2016/States')

## Weights for each dataset

In [4]:
W_IMP = 3
W_SMP = 4
W_PREC = 5
W_ELEV = 1
W_FLD1 = 2
W_FLD2 = 3
palette = ['fee5d9', 'fcae91', 'fb6a4a', 'de2d26', 'de2d26', 'a50f15', 'a50f15', 'a50f15']

## Map and scale impervious surface percentage from USGS NLCD

In [5]:
# Import the USGS National Land Cover Database image collection.
impervious = ee.ImageCollection("USGS/NLCD_RELEASES/2016_REL").filterDate('2016-01-01', '2017-01-01').select('impervious').median()
params = {'min': 0, 'max': 100, 'palette': ['ffffcc','c7e9b4','7fcdbb','41b6c4','2c7fb8','253494']}
#Map.addLayer(impervious, params, 'Impervious Surface Percentage 2016 USGS NLCD')

impervious_s = impervious.divide(ee.Image.constant(10/W_IMP))
params2 = {'min': 0, 'max': 10*W_IMP, 'palette': palette}
#Map.addLayer(impervious_s.selfMask(), params2, 'Scaled Impervious Surface Percentage 2016 USGS NLCD')

## Map and scale soil moisture profile from the SMAP dataset

In [6]:
smp = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture") \
    .filterDate('2020-11-23T00:00', '2021-11-23T0:00') \
    .select('smp') \
    .median() \
    .clip(aoi)
smp_old = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture") \
    .filterDate('2015-04-01T00:00', '2016-04-01T0:00') \
    .select('smp') \
    .median() \
    .clip(aoi)
smpVis = {'min': 0, 'max': 1, 'palette': ['0300ff', '418504', 'efff07', 'efff07', 'ff0303']}
#Map.addLayer(smp_old, smpVis, 'Old Soil Moisture')
#Map.addLayer(smp, smpVis, 'Soil Moisture')

smpVis2 = {'min': 0, 'max': 10*W_SMP, 'palette': palette}
smp_s = smp.multiply(ee.Image.constant(W_SMP*10))
#Map.addLayer(smp_s.selfMask(), smpVis2, 'Scaled Soil Moisture')

## Map and precipitation data from the IMERG dataset





In [7]:
precipitation = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").filterDate('2020-07-01T08:00', '2021-07-01T08:00').select('precipitation').max().clip(aoi)
precipitation_old = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").filterDate('2000-06-01T00:00', '2001-06-01T00:00').select('precipitation').max().clip(aoi)
imerg_palette = ['000096','0064ff', '00b4ff', '33db80', '9beb4a', 'ffeb00', 'ffb300', 'ff6400', 'eb1e00', 'af0000']
imerg_params = {'min': 0, 'max': 1.5, 'palette': imerg_palette}
#Map.addLayer(precipitation_old, imerg_params, 'Old IMERG Precipitation')
#Map.addLayer(precipitation, imerg_params, 'IMERG Precipitation')

imerg_params2 = {'min': 0, 'max': 10*W_PREC, 'palette': palette}
precipitation_s = precipitation.multiply(ee.Image.constant(W_PREC*(10/1.5)))
#Map.addLayer(precipitation_s.selfMask(), imerg_params2, 'Scaled IMERG Precipitation 2017')

## Map and elevation data





In [8]:
dataset = ee.Image('NASA/NASADEM_HGT/001')
elevation = dataset.select('elevation').clip(aoi)

elevationVis = {
  'min': 0,
  'max': 2000,
  'palette': ['white', 'blue']
}

#Map.addLayer(elevation.updateMask(elevation.gt(0)), elevationVis, 'Elevation')
#Map.addLayer(ee.Image.constant(2000).subtract(elevation.updateMask(elevation.gt(0))), elevationVis, 'Elevation Reversed')

elevationVis2 = {
  'min': 0,
  'max': 10*W_ELEV,
  'palette': palette
}
elevation_s = elevation.multiply(ee.Image.constant(-1)).add(ee.Image.constant(2000)).divide(ee.Image.constant(200/W_ELEV))
#Map.addLayer(elevation_s.selfMask(), elevationVis2, 'Scaled Elevation')


## Map water during floods from the Global Flood Database


In [9]:
gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1').map(lambda x: x.clip(aoi))
gfdFloodedSum = gfd.select('flooded').sum()
'''Map.addLayer(
  gfdFloodedSum,
  {'min': 0, 'max': 10, 'palette': ['eff3ff','bdd7e7','6baed6','2171b5']},
  'GFD Satellite Observed Flood Plain')'''

"Map.addLayer(\n  gfdFloodedSum,\n  {'min': 0, 'max': 10, 'palette': ['eff3ff','bdd7e7','6baed6','2171b5']},\n  'GFD Satellite Observed Flood Plain')"

## Isolate areas of flooding
The 'flooded' band includes permanent water. Therefore, areas of permanent water must be subtracted from areas where there is water during floods. This can be done using the 'jrc_perm_water' band.

In [10]:
jrc = gfd.select('jrc_perm_water').sum().gte(1)
'''Map.addLayer(
  jrc.selfMask(),
  {'min': 0, 'max': 1, 'palette': '2171b5'},
  'JRC Permanent Water')'''
flood1 = gfdFloodedSum.multiply(jrc.lt(1)).multiply(ee.Image.constant(W_FLD1))
#Map.addLayer(flood1.selfMask(), {'min': 0, 'max': 10*W_FLD1, 'palette': palette}, 'Scaled Flooded Areas')
#flood_no_perm = flood_no_perm.reproject(floodProjection, None, 30)
#flood_no_perm = flood_no_perm.reproject(smpProjection, None, 500)
#.reduceResolution(reducer=ee.Reducer.max(), maxPixels=2000)
#flood_no_perm = flood_no_perm.reproject(crs=precipitation.projection(), scale=10000).reduceResolution(reducer=ee.Reducer.max(), maxPixels=1024)
flood2 = gfdFloodedSum.resample('bilinear').multiply(jrc.lt(1)).multiply(ee.Image.constant(W_FLD2))
#Map.addLayer(flood2.selfMask(), {'min': 0, 'max': 10*W_FLD2, 'palette': palette}, 'Resampled Scaled Flooded Areas')

In [11]:
#bg = ee.Image.constant(0).clip(aoi)
#Map.addLayer(bg, {'min': 0, 'max': 0, 'palette': 'white'})
combined = ee.Image().clip(aoi)
combined = combined.expression(
      'prec+imp+smp+flood1+flood2+elev+bg',
      {
      'prec': precipitation_s.select('precipitation').unmask(),
      'imp': impervious_s.select('impervious').unmask(),
      'smp' : smp_s.select('smp').unmask(),
      'flood1' : flood1.select('flooded').unmask(),
      'flood2' : flood2.select('flooded').unmask(),
      'elev' : elevation_s.select('elevation').unmask(),
      'bg': ee.Image.constant(0)

      }).rename('RISK')
combined = combined.select('RISK')
combined = combined.updateMask(combined.neq(0))
Map.addLayer(combined, {'min': 0, 'max': 10*(W_PREC+W_IMP+W_SMP+W_FLD1+W_FLD2+W_ELEV), 'palette': palette}, 'Combined')

## Display Map with all layers


In [12]:
# legend_dict = dict(zip(['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0'], ['0300ff', '418504', 'efff07', 'efff07', 'ff0303']))
# Map.add_legend(legend_dict=legend_dict, position='bottomleft')

Map.addLayer(
  jrc.selfMask(),
  {'min': 0, 'max': 1, 'palette': 'C3EFFE'},
  'JRC Permanent Water')

Map

Map(center=[23.182838, -75.722793], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…